In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# tiger

In [3]:
# 讀取 CSV 檔案，請將 'your_file.csv' 替換成你的檔案名稱
file_path = "./data/tiger_label.csv"
df_reviews = pd.read_csv(file_path)

df_reviews

,Review Id,Published Date,Language,Location,Sentence Id,Sentence,sentiment,Label
0,983008103,2024-12-02,ko,tigerair taiwan,2,Failed to board the plane.,-0.544636,Label_3
1,983008103,2024-12-02,ko,tigerair taiwan,3,Team 2 arrived 8 minutes before boarding and w...,-0.134854,Label_3
2,983008103,2024-12-02,ko,tigerair taiwan,9,"Since it is a low-cost airline, it does not br...",-0.114661,Label_4
3,983008103,2024-12-02,ko,tigerair taiwan,10,"Because it is a low-cost airline, even the fro...",-0.697315,Label_2
4,983008103,2024-12-02,ko,tigerair taiwan,11,"Since it is a low-cost airline, the boarding p...",-0.063565,Label_3
...,...,...,...,...,...,...,...,...
923,971661772,2024-09-27,ja,tigerair taiwan,9,"A short time after leaving, food was distributed.",0.281590,Label_4
924,971661772,2024-09-27,ja,tigerair taiwan,9,"A short time after leaving, food was distributed.",0.281590,Label_5
925,971661772,2024-09-27,ja,tigerair taiwan,10,It's pork and rice.,0.361717,Label_5
926,971661772,2024-09-27,ja,tigerair taiwan,13,"There is a charge for alcohol on board, so I a...",-0.112176,Label_3


In [4]:
# 讀取 CSV 檔案，請將 'your_file.csv' 替換成你的檔案名稱
file_path = "./0909/tiger_event_true_5_39.csv"
df_events = pd.read_csv(file_path)
# 移除任何含有 NaN 的列（預設 axis=0 表示 row）
df_events = df_events.dropna()

df_events

,Published Date,Event
0,2019/1/18,Event_5
1,2019/2/4,Event_4
2,2019/2/19,Event_6
3,2019/2/20,Event_1
4,2019/3/3,Event_5
5,2019/3/9,Event_3
6,2019/3/27,Event_1
7,2019/4/3,Event_6
8,2019/4/16,Event_7
9,2019/4/21,Event_7


## [0]

In [5]:
# 確保格式正確
df_reviews['Published Date'] = pd.to_datetime(df_reviews['Published Date'])
df_events['Published Date'] = pd.to_datetime(df_events['Published Date'])

# 儲存結果
results = []

# 每一筆事件處理
for _, row in df_events.iterrows():
    event_date = row['Published Date']
    event_label = row['Event']  # e.g., 'Event_3'
    
    # 提取數字，例如 'Event_3' ➜ 3
    match = re.search(r'Event_(\d+)', event_label)
    if not match:
        continue  # 跳過無效事件名稱
    
    label_index = int(match.group(1))
    label_name = f'Label_{label_index}'
    
    # 篩出當天該分類的評論
    day_data = df_reviews[
        (df_reviews['Published Date'] == event_date) & 
        (df_reviews['Label'] == label_name)
    ]
    
    # 統計欄位
    total_sentences = len(day_data)
    avg_sentiment = day_data['sentiment'].mean() if total_sentences > 0 else 0

    pos_sentiments = day_data[day_data['sentiment'] > 0]['sentiment']
    neg_sentiments = day_data[day_data['sentiment'] < 0]['sentiment']

    result_row = {
        'Event Date': event_date,
        'Event': event_label,
        'Label': label_name,
        'Count': total_sentences,
        'Avg Sentiment': avg_sentiment,
        'Avg Positive Sentiment': pos_sentiments.mean() if not pos_sentiments.empty else 0,
        'Avg Negative Sentiment': neg_sentiments.mean() if not neg_sentiments.empty else 0,
        'Positive Count': pos_sentiments.count(),
        'Negative Count': neg_sentiments.count()
    }

    results.append(result_row)

# 整理成表格
df_event_label_stats = pd.DataFrame(results)


In [6]:
df_event_label_stats

,Event Date,Event,Label,Count,Avg Sentiment,Avg Positive Sentiment,Avg Negative Sentiment,Positive Count,Negative Count
0,2019-01-18,Event_5,Label_5,1,-0.246926,0.000000,-0.246926,0,1
1,2019-02-04,Event_4,Label_4,5,0.119179,0.378001,-0.269053,3,2
2,2019-02-19,Event_6,Label_6,2,0.521817,0.521817,0.000000,2,0
3,2019-02-20,Event_1,Label_1,2,0.468557,0.468557,0.000000,2,0
4,2019-03-03,Event_5,Label_5,1,0.771814,0.771814,0.000000,1,0
5,2019-03-09,Event_3,Label_3,1,0.373970,0.373970,0.000000,1,0
6,2019-03-27,Event_1,Label_1,1,0.745568,0.745568,0.000000,1,0
7,2019-04-03,Event_6,Label_6,2,-0.120168,0.084893,-0.325229,1,1
8,2019-04-16,Event_7,Label_7,1,0.042635,0.042635,0.000000,1,0
9,2019-04-21,Event_7,Label_7,2,-0.070442,0.000000,-0.070442,0,2


In [8]:
df_event_label_stats.to_csv("./0909/tiger_feature_5.csv")

## [-1:1]

In [10]:
# 確保日期格式正確
df_reviews['Published Date'] = pd.to_datetime(df_reviews['Published Date'])
df_events['Published Date'] = pd.to_datetime(df_events['Published Date'])

# 儲存結果
results = []

# 每一筆事件處理
for _, row in df_events.iterrows():
    event_date = row['Published Date']
    event_label = row['Event']  # e.g., 'Event_3'

    # 提取數字：Event_3 ➜ 3
    match = re.search(r'Event_(\d+)', event_label)
    if not match:
        continue  # 跳過不合法的事件

    label_index = int(match.group(1))
    label_name = f'Label_{label_index}'

    # 抓事件日前後一天：[-1, 0, +1]
    mask = (
        (df_reviews['Published Date'] >= event_date - pd.Timedelta(days=1)) &
        (df_reviews['Published Date'] <= event_date + pd.Timedelta(days=1)) &
        (df_reviews['Label'] == label_name)
    )
    window_data = df_reviews[mask]

    # 統計
    total_sentences = len(window_data)
    avg_sentiment = window_data['sentiment'].mean() if total_sentences > 0 else 0

    pos_sentiments = window_data[window_data['sentiment'] > 0]['sentiment']
    neg_sentiments = window_data[window_data['sentiment'] < 0]['sentiment']

    result_row = {
        'Event Date': event_date,
        'Event': event_label,
        'Label': label_name,
        'Count[-1:+1]': total_sentences,
        'Avg Sentiment[-1:+1]': avg_sentiment,
        'Avg Positive Sentiment[-1:+1]': pos_sentiments.mean() if not pos_sentiments.empty else 0,
        'Avg Negative Sentiment[-1:+1]': neg_sentiments.mean() if not neg_sentiments.empty else 0,
        'Positive Count[-1:+1]': pos_sentiments.count(),
        'Negative Count[-1:+1]': neg_sentiments.count()
    }

    results.append(result_row)

# 轉成 DataFrame
df_event_window_stats = pd.DataFrame(results)

df_event_window_stats

,Event Date,Event,Label,Count[-1:+1],Avg Sentiment[-1:+1],Avg Positive Sentiment[-1:+1],Avg Negative Sentiment[-1:+1],Positive Count[-1:+1],Negative Count[-1:+1]
0,2019-01-18,Event_5,Label_5,1,-0.246926,0.000000,-0.246926,0,1
1,2019-02-04,Event_4,Label_4,5,0.119179,0.378001,-0.269053,3,2
2,2019-02-19,Event_6,Label_6,3,0.438965,0.438965,0.000000,3,0
3,2019-02-20,Event_1,Label_1,2,0.468557,0.468557,0.000000,2,0
4,2019-03-03,Event_5,Label_5,1,0.771814,0.771814,0.000000,1,0
5,2019-03-09,Event_3,Label_3,9,-0.269842,0.174836,-0.492182,3,6
6,2019-03-27,Event_1,Label_1,1,0.745568,0.745568,0.000000,1,0
7,2019-04-03,Event_6,Label_6,2,-0.120168,0.084893,-0.325229,1,1
8,2019-04-16,Event_7,Label_7,1,0.042635,0.042635,0.000000,1,0
9,2019-04-21,Event_7,Label_7,2,-0.070442,0.000000,-0.070442,0,2


In [11]:
df_event_window_stats.to_csv("./0909/tiger_feature_-1to1_5.csv")

# eva

In [12]:
# 讀取 CSV 檔案，請將 'your_file.csv' 替換成你的檔案名稱
file_path = "./data/eva_label.csv"
df_reviews = pd.read_csv(file_path)

df_reviews

,Review Id,Published Date,Language,Location,Sentence Id,Sentence,sentiment,Label
0,645471677,2019-01-13,de,EVA Air,3,"great delicious food, you got blankets and sli...",0.923588,Label_5
1,646999582,2019-01-19,de,EVA Air,3,From Berlin - change in London - to Bangkok.,0.276517,Label_1
2,646999582,2019-01-19,de,EVA Air,7,Due to the delay a lot of running back and for...,-0.133133,Label_3
3,646999582,2019-01-19,de,EVA Air,8,We reach this wet sweat 2 minutes before the a...,0.011587,Label_3
4,646999582,2019-01-19,de,EVA Air,9,"On board, we wanted to see the on-board progra...",0.275575,Label_3
...,...,...,...,...,...,...,...,...
11212,966759164,2024-08-25,zhTW,EVA Air,9,"At the end of the day, it's all about money.",-0.123406,Label_7
11213,968198705,2024-09-03,zhTW,EVA Air,2,"When waiting for upgrading, the original booki...",-0.315015,Label_1
11214,968198705,2024-09-03,zhTW,EVA Air,5,"~ If you don't make up for it, the mileage is ...",-0.692617,Label_4
11215,968198705,2024-09-03,zhTW,EVA Air,7,Since tourists are encouraged to choose Evergr...,0.778116,Label_1


In [14]:
# 讀取 CSV 檔案，請將 'your_file.csv' 替換成你的檔案名稱
file_path = "./0909/eva_event_true_5_185.csv"
df_events = pd.read_csv(file_path)
# 移除任何含有 NaN 的列（預設 axis=0 表示 row）
df_events = df_events.dropna()

df_events

,Published Date,Event
0,2019/4/22,Event_1
1,2019/5/13,Event_1
2,2019/7/5,Event_1
3,2019/10/29,Event_1
4,2020/1/18,Event_1
...,...,...
180,2024/3/17,Event_7
181,2024/5/2,Event_7
182,2024/5/19,Event_7
183,2024/10/17,Event_7


## [0]

In [15]:
# 確保格式正確
df_reviews['Published Date'] = pd.to_datetime(df_reviews['Published Date'])
df_events['Published Date'] = pd.to_datetime(df_events['Published Date'])

# 儲存結果
results = []

# 每一筆事件處理
for _, row in df_events.iterrows():
    event_date = row['Published Date']
    event_label = row['Event']  # e.g., 'Event_3'
    
    # 提取數字，例如 'Event_3' ➜ 3
    match = re.search(r'Event_(\d+)', event_label)
    if not match:
        continue  # 跳過無效事件名稱
    
    label_index = int(match.group(1))
    label_name = f'Label_{label_index}'
    
    # 篩出當天該分類的評論
    day_data = df_reviews[
        (df_reviews['Published Date'] == event_date) & 
        (df_reviews['Label'] == label_name)
    ]
    
    # 統計欄位
    total_sentences = len(day_data)
    avg_sentiment = day_data['sentiment'].mean() if total_sentences > 0 else 0

    pos_sentiments = day_data[day_data['sentiment'] > 0]['sentiment']
    neg_sentiments = day_data[day_data['sentiment'] < 0]['sentiment']

    result_row = {
        'Event Date': event_date,
        'Event': event_label,
        'Label': label_name,
        'Count': total_sentences,
        'Avg Sentiment': avg_sentiment,
        'Avg Positive Sentiment': pos_sentiments.mean() if not pos_sentiments.empty else 0,
        'Avg Negative Sentiment': neg_sentiments.mean() if not neg_sentiments.empty else 0,
        'Positive Count': pos_sentiments.count(),
        'Negative Count': neg_sentiments.count()
    }

    results.append(result_row)

# 整理成表格
df_event_label_stats = pd.DataFrame(results)


In [16]:
df_event_label_stats.to_csv("./0909/eva_feature_5.csv")
df_event_label_stats

,Event Date,Event,Label,Count,Avg Sentiment,Avg Positive Sentiment,Avg Negative Sentiment,Positive Count,Negative Count
0,2019-04-22,Event_1,Label_1,6,0.335744,0.335744,0.000000,6,0
1,2019-05-13,Event_1,Label_1,9,0.030371,0.555381,-0.232135,3,6
2,2019-07-05,Event_1,Label_1,3,-0.120325,0.261986,-0.311481,1,2
3,2019-10-29,Event_1,Label_1,4,0.656497,0.656497,0.000000,4,0
4,2020-01-18,Event_1,Label_1,4,-0.176646,0.000000,-0.176646,0,4
...,...,...,...,...,...,...,...,...,...
180,2024-03-17,Event_7,Label_7,2,0.297981,0.297981,0.000000,2,0
181,2024-05-02,Event_7,Label_7,2,-0.075051,0.000000,-0.075051,0,2
182,2024-05-19,Event_7,Label_7,3,-0.292497,0.010640,-0.444066,1,2
183,2024-10-17,Event_7,Label_7,1,0.625700,0.625700,0.000000,1,0


## [-1:1]

In [17]:
# 確保日期格式正確
df_reviews['Published Date'] = pd.to_datetime(df_reviews['Published Date'])
df_events['Published Date'] = pd.to_datetime(df_events['Published Date'])

# 儲存結果
results = []

# 每一筆事件處理
for _, row in df_events.iterrows():
    event_date = row['Published Date']
    event_label = row['Event']  # e.g., 'Event_3'

    # 提取數字：Event_3 ➜ 3
    match = re.search(r'Event_(\d+)', event_label)
    if not match:
        continue  # 跳過不合法的事件

    label_index = int(match.group(1))
    label_name = f'Label_{label_index}'

    # 抓事件日前後一天：[-1, 0, +1]
    mask = (
        (df_reviews['Published Date'] >= event_date - pd.Timedelta(days=1)) &
        (df_reviews['Published Date'] <= event_date + pd.Timedelta(days=1)) &
        (df_reviews['Label'] == label_name)
    )
    window_data = df_reviews[mask]

    # 統計
    total_sentences = len(window_data)
    avg_sentiment = window_data['sentiment'].mean() if total_sentences > 0 else 0

    pos_sentiments = window_data[window_data['sentiment'] > 0]['sentiment']
    neg_sentiments = window_data[window_data['sentiment'] < 0]['sentiment']

    result_row = {
        'Event Date': event_date,
        'Event': event_label,
        'Label': label_name,
        'Count[-1:+1]': total_sentences,
        'Avg Sentiment[-1:+1]': avg_sentiment,
        'Avg Positive Sentiment[-1:+1]': pos_sentiments.mean() if not pos_sentiments.empty else 0,
        'Avg Negative Sentiment[-1:+1]': neg_sentiments.mean() if not neg_sentiments.empty else 0,
        'Positive Count[-1:+1]': pos_sentiments.count(),
        'Negative Count[-1:+1]': neg_sentiments.count()
    }

    results.append(result_row)

# 轉成 DataFrame
df_event_window_stats = pd.DataFrame(results)

df_event_window_stats

,Event Date,Event,Label,Count[-1:+1],Avg Sentiment[-1:+1],Avg Positive Sentiment[-1:+1],Avg Negative Sentiment[-1:+1],Positive Count[-1:+1],Negative Count[-1:+1]
0,2019-04-22,Event_1,Label_1,10,0.380301,0.453941,-0.282456,9,1
1,2019-05-13,Event_1,Label_1,12,0.117868,0.569830,-0.204962,5,7
2,2019-07-05,Event_1,Label_1,6,0.280459,0.576429,-0.311481,4,2
3,2019-10-29,Event_1,Label_1,10,0.362091,0.714413,-0.459992,7,3
4,2020-01-18,Event_1,Label_1,5,-0.075846,0.327352,-0.176646,1,4
...,...,...,...,...,...,...,...,...,...
180,2024-03-17,Event_7,Label_7,5,-0.013473,0.212187,-0.351963,3,2
181,2024-05-02,Event_7,Label_7,2,-0.075051,0.000000,-0.075051,0,2
182,2024-05-19,Event_7,Label_7,5,-0.055829,0.202995,-0.444066,3,2
183,2024-10-17,Event_7,Label_7,1,0.625700,0.625700,0.000000,1,0


In [18]:
df_event_window_stats.to_csv("./0909/eva_feature_-1to1_5.csv")

# china

In [19]:
# 讀取 CSV 檔案，請將 'your_file.csv' 替換成你的檔案名稱
file_path = "./data/china_label.csv"
df_reviews = pd.read_csv(file_path)

df_reviews

,Review Id,Published Date,Language,Location,Sentence Id,Sentence,sentiment,Label
0,643662856,2019-01-04,de,China Airlines,1,Quite ok. Seat pitch ok. Food good.,0.916222,Label_6
1,643662856,2019-01-04,de,China Airlines,2,Lack of knowledge of the staff.,0.250197,Label_5
2,643662856,2019-01-04,de,China Airlines,4,Drinks often and well.,0.318024,Label_5
3,643662856,2019-01-04,de,China Airlines,5,Staff friendly,0.557128,Label_5
4,644717716,2019-01-09,de,China Airlines,2,"Punctual, friendly, decent catering and exit-r...",0.705783,Label_4
...,...,...,...,...,...,...,...,...
5910,976905019,2024-10-27,zhTW,China Airlines,1,Ticket Changes.,0.061147,Label_1
5911,976905019,2024-10-27,zhTW,China Airlines,3,Call customer service and say you can change y...,0.250397,Label_1
5912,976905019,2024-10-27,zhTW,China Airlines,4,"There is no handling fee, but the same price i...",0.188398,Label_1
5913,976905019,2024-10-27,zhTW,China Airlines,7,Ticket changes are possible.,0.217205,Label_1


In [22]:
# 讀取 CSV 檔案，請將 'your_file.csv' 替換成你的檔案名稱
file_path = "./0909/china_event_true_5_138.csv"
df_events = pd.read_csv(file_path)

# 移除任何含有 NaN 的列（預設 axis=0 表示 row）
df_events = df_events.dropna()

df_events

,Published Date,Event
0,2019/1/6,Event_4
1,2019/1/9,Event_4
2,2019/1/10,Event_7
3,2019/1/13,Event_6
4,2019/1/17,Event_7
...,...,...
133,2024/11/19,Event_6
134,2024/12/16,Event_4
135,2024/12/22,Event_5
136,2024/12/24,Event_7


## [0]　

In [25]:
# 確保格式正確
df_reviews['Published Date'] = pd.to_datetime(df_reviews['Published Date'])
df_events['Published Date'] = pd.to_datetime(df_events['Published Date'])

# 儲存結果
results = []

# 每一筆事件處理
for _, row in df_events.iterrows():
    event_date = row['Published Date']
    event_label = row['Event']  # e.g., 'Event_3'
    
    # 提取數字，例如 'Event_3' ➜ 3
    match = re.search(r'Event_(\d+)', event_label)
    if not match:
        continue  # 跳過無效事件名稱
    
    label_index = int(match.group(1))
    label_name = f'Label_{label_index}'
    
    # 篩出當天該分類的評論
    day_data = df_reviews[
        (df_reviews['Published Date'] == event_date) & 
        (df_reviews['Label'] == label_name)
    ]
    
    # 統計欄位
    total_sentences = len(day_data)
    avg_sentiment = day_data['sentiment'].mean() if total_sentences > 0 else 0

    pos_sentiments = day_data[day_data['sentiment'] > 0]['sentiment']
    neg_sentiments = day_data[day_data['sentiment'] < 0]['sentiment']

    result_row = {
        'Event Date': event_date,
        'Event': event_label,
        'Label': label_name,
        'Count': total_sentences,
        'Avg Sentiment': avg_sentiment,
        'Avg Positive Sentiment': pos_sentiments.mean() if not pos_sentiments.empty else 0,
        'Avg Negative Sentiment': neg_sentiments.mean() if not neg_sentiments.empty else 0,
        'Positive Count': pos_sentiments.count(),
        'Negative Count': neg_sentiments.count()
    }

    results.append(result_row)

# 整理成表格
df_event_label_stats = pd.DataFrame(results)


In [26]:
df_event_label_stats.to_csv("./0909/china_feature_5.csv")
df_event_label_stats

,Event Date,Event,Label,Count,Avg Sentiment,Avg Positive Sentiment,Avg Negative Sentiment,Positive Count,Negative Count
0,2019-01-06,Event_4,Label_4,8,0.256776,0.510823,-0.505362,6,2
1,2019-01-09,Event_4,Label_4,10,0.453245,0.730701,-0.194153,7,3
2,2019-01-10,Event_7,Label_7,7,0.316020,0.316020,0.000000,7,0
3,2019-01-13,Event_6,Label_6,7,0.582041,0.582041,0.000000,7,0
4,2019-01-17,Event_7,Label_7,4,0.334047,0.509335,-0.191817,3,1
...,...,...,...,...,...,...,...,...,...
133,2024-11-19,Event_6,Label_6,3,0.307746,0.307746,0.000000,3,0
134,2024-12-16,Event_4,Label_4,3,0.032942,0.058335,-0.017842,2,1
135,2024-12-22,Event_5,Label_5,4,0.033863,0.376731,-0.309005,2,2
136,2024-12-24,Event_7,Label_7,3,-0.292405,0.010225,-0.443720,1,2


## [-1:1]

In [27]:
# 確保日期格式正確
df_reviews['Published Date'] = pd.to_datetime(df_reviews['Published Date'])
df_events['Published Date'] = pd.to_datetime(df_events['Published Date'])

# 儲存結果
results = []

# 每一筆事件處理
for _, row in df_events.iterrows():
    event_date = row['Published Date']
    event_label = row['Event']  # e.g., 'Event_3'

    # 提取數字：Event_3 ➜ 3
    match = re.search(r'Event_(\d+)', event_label)
    if not match:
        continue  # 跳過不合法的事件

    label_index = int(match.group(1))
    label_name = f'Label_{label_index}'

    # 抓事件日前後一天：[-1, 0, +1]
    mask = (
        (df_reviews['Published Date'] >= event_date - pd.Timedelta(days=1)) &
        (df_reviews['Published Date'] <= event_date + pd.Timedelta(days=1)) &
        (df_reviews['Label'] == label_name)
    )
    window_data = df_reviews[mask]

    # 統計
    total_sentences = len(window_data)
    avg_sentiment = window_data['sentiment'].mean() if total_sentences > 0 else 0

    pos_sentiments = window_data[window_data['sentiment'] > 0]['sentiment']
    neg_sentiments = window_data[window_data['sentiment'] < 0]['sentiment']

    result_row = {
        'Event Date': event_date,
        'Event': event_label,
        'Label': label_name,
        'Count[-1:+1]': total_sentences,
        'Avg Sentiment[-1:+1]': avg_sentiment,
        'Avg Positive Sentiment[-1:+1]': pos_sentiments.mean() if not pos_sentiments.empty else 0,
        'Avg Negative Sentiment[-1:+1]': neg_sentiments.mean() if not neg_sentiments.empty else 0,
        'Positive Count[-1:+1]': pos_sentiments.count(),
        'Negative Count[-1:+1]': neg_sentiments.count()
    }

    results.append(result_row)

# 轉成 DataFrame
df_event_window_stats = pd.DataFrame(results)

df_event_window_stats

,Event Date,Event,Label,Count[-1:+1],Avg Sentiment[-1:+1],Avg Positive Sentiment[-1:+1],Avg Negative Sentiment[-1:+1],Positive Count[-1:+1],Negative Count[-1:+1]
0,2019-01-06,Event_4,Label_4,12,0.160746,0.416532,-0.350826,8,4
1,2019-01-09,Event_4,Label_4,17,0.380616,0.665763,-0.303739,12,5
2,2019-01-10,Event_7,Label_7,9,0.322277,0.377349,-0.118293,8,1
3,2019-01-13,Event_6,Label_6,7,0.582041,0.582041,0.000000,7,0
4,2019-01-17,Event_7,Label_7,14,0.142368,0.438900,-0.253007,8,6
...,...,...,...,...,...,...,...,...,...
133,2024-11-19,Event_6,Label_6,3,0.307746,0.307746,0.000000,3,0
134,2024-12-16,Event_4,Label_4,3,0.032942,0.058335,-0.017842,2,1
135,2024-12-22,Event_5,Label_5,5,-0.052120,0.376731,-0.338021,2,3
136,2024-12-24,Event_7,Label_7,3,-0.292405,0.010225,-0.443720,1,2


In [28]:
df_event_window_stats.to_csv("./0909/china_feature_-1to1_5.csv")